### Import modules

First of all, we'll import all useful modules to our notebook.

In [86]:
#Import modules
%matplotlib inline

#handling data
import pandas as pd
import numpy as np

#import twitter credentials
from twitter_credentials import *

#handling information
import re
import json

#handling plots
import matplotlib.pyplot as plt

#for network creation
import networkx as nx

### Read json to DataFrame

The information that we've collected is stored in the file 'tweets.txt'. Because this file has a JSON format, we'll take advantage of the `read_json` function of the pandas module. 

In [87]:
pd.set_option('display.float_format', lambda x: '%.f' % x)

In [88]:
# Read json into a pandas dataframe
tweets_df = pd.read_json("tweets.txt", lines=True)

According to [Twitter API website](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object), the Tweet object retrieved, provided in JSON format, has a long list of mixed `root-level` attributes, including basic information such as `id`, `created_at`, and `text`. Tweet objects are also the `parent` object to several child objects. Tweet child objects include `user`, `entities`, and extended_entities.

In order to have a better idea of the information we are dealing with, let's take a look at the `DataFrame` columns.

In [89]:
# Let's check the name of the columns
tweets_df.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'reply_count', 'retweet_count', 'retweeted', 'retweeted_status',
       'source', 'text', 'timestamp_ms', 'truncated', 'user'],
      dtype='object')

From the displayed columns, we can observe some of them that look interesting for our future analysis. If we look more in detail, certain columns, such as `retweeted_status`, `entities` provide us with information about interactions regarding user mentions and user retweets. So, we are going to create a new `DataFrame` where to store all this important information that will come in handy when we build our Network. 

In [5]:
# Create a second dataframe to put important information
tweets_final = pd.DataFrame(columns = ["created_at", "id", "in_reply_to_screen_name", "in_reply_to_status_id", "in_reply_to_user_id",
                                      "retweeted_id", "retweeted_screen_name", "user_mentions_screen_name", "user_mentions_id", 
                                       "text", "user_id", "screen_name", "followers_count"])

In [6]:
# Columns that are going to be the same
equal_columns = ["created_at", "id", "text"]
tweets_final[equal_columns] = tweets_df[equal_columns]

Now, we need to obtain important information. There are three types of interactions between two Twitter users that we should gather: retweets, replies, and mentions.

From the structure of the [JSON file](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/intro-to-tweet-json#tweetobject) retrieved by the Twitter API, we learn that we can obtain basic information about the user: 

In [7]:
# Get the basic information about user 
def get_basics(tweets_final):
    tweets_final["screen_name"] = tweets_df["user"].apply(lambda x: x["screen_name"])
    tweets_final["user_id"] = tweets_df["user"].apply(lambda x: x["id"])
    tweets_final["followers_count"] = tweets_df["user"].apply(lambda x: x["followers_count"])
    return tweets_final

The `entities` object is a dictionary that contains, among other information, the `user_mentions` object that represents other Twitter users (`screen_name` and `id`) mentioned in the text of the Tweet.  

In [8]:
# Get the user mentions 
def get_usermentions(tweets_final):
    # Inside the tag 'entities' will find 'user mentions' and will get 'screen name' and 'id'
    tweets_final["user_mentions_screen_name"] = tweets_df["entities"].apply(lambda x: x["user_mentions"][0]["screen_name"] if x["user_mentions"] else np.nan)
    tweets_final["user_mentions_id"] = tweets_df["entities"].apply(lambda x: x["user_mentions"][0]["id_str"] if x["user_mentions"] else np.nan)
    return tweets_final

The retweets always contain two Tweet objects: The original Tweet being Retweeted is provided in a `retweeted_status` object. Inside, the `user` object for the account (`screen_name` and `id`) taking the Retweet action and the time of the Retweet.  

In [9]:
# Get retweets
def get_retweets(tweets_final):
    # Inside the tag 'retweeted_status' will find 'user' and will get 'screen name' and 'id'    
    tweets_final["retweeted_screen_name"] = tweets_df["retweeted_status"].apply(lambda x: x["user"]["screen_name"] if x is not np.nan else np.nan)
    tweets_final["retweeted_id"] = tweets_df["retweeted_status"].apply(lambda x: x["user"]["id_str"] if x is not np.nan else np.nan)
    return tweets_final

From the Tweet object, we can get the information of the user and status to which the tweet replies to:

In [10]:
# Get the information about replies
def get_in_reply(tweets_final):
    # Just copy the 'in_reply' columns to the new dataframe
    tweets_final["in_reply_to_screen_name"] = tweets_df["in_reply_to_screen_name"]
    tweets_final["in_reply_to_status_id"] = tweets_df["in_reply_to_status_id"]
    tweets_final["in_reply_to_user_id"]= tweets_df["in_reply_to_user_id"]
    return tweets_final

Now, we define a function that calls all the functions to fill the DataFrame with the usefull variables.

In [11]:
# Lastly fill the new dataframe with the important information
def fill_df(tweets_final):
    get_basics(tweets_final)
    get_usermentions(tweets_final)
    get_retweets(tweets_final)
    get_in_reply(tweets_final)
    return tweets_final

It's time to get from the DataFrame all the interactions. For this aim, we'll define a function that will iterate over the DataFrame, obtain the user_id and screen_name of the user that the author of that specific tweet mention, reply or retweet. The function will return the user of the specific tweet together with a list of the users with whom the user interacted.

In [12]:
# Get the interactions between the different users
def get_interactions(row):
    # From every row of the original dataframe
    # First we obtain the 'user_id' and 'screen_name'
    user = row["user_id"], row["screen_name"]
    # Be careful if there is no user id
    if user[0] is None:
        return (None, None), []
    
    # The interactions are going to be a set of tuples
    interactions = set()
    
    # Add all interactions 
    # First, we add the interactions corresponding to replies adding the id and screen_name
    interactions.add((row["in_reply_to_user_id"], row["in_reply_to_screen_name"]))
    # After that, we add the interactions with retweets
    interactions.add((row["retweeted_id"], row["retweeted_screen_name"]))
    # And later, the interactions with user mentions
    interactions.add((row["user_mentions_id"], row["user_mentions_screen_name"]))
    
    # Discard if user id is in interactions
    interactions.discard((row["user_id"], row["screen_name"]))
    # Discard all not existing values
    interactions.discard((None, None))
    # Return user and interactions
    return user, interactions

Now, we fill the DataFrame and discard the None values.

In [13]:
tweets_final = fill_df(tweets_final)

Be aware of None values!

In [14]:
tweets_final = tweets_final.where((pd.notnull(tweets_final)), None)

In [90]:
tweets_final.head(5)

,created_at,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweeted_id,retweeted_screen_name,user_mentions_screen_name,user_mentions_id,text,user_id,screen_name,followers_count
0,2019-03-30 20:14:37,1112085741683638272,None,None,None,938373003355635712,4fnan_zainal,4fnan_zainal,938373003355635712,RT @4fnan_zainal: Pls ban kids masa showtime a...,925075656127365120,iRfansyamill,39
1,2019-03-30 20:14:39,1112085751267762177,None,None,None,257265441,skipbolden,skipbolden,257265441,RT @skipbolden: The Avengers (TV Series 1961–1...,812752256214925312,Brianlovesmovi,5906
2,2019-03-30 20:14:41,1112085758851059714,None,None,None,None,None,None,None,Looking forward to the Avengers End Game 😍 #av...,924977392535339010,GadgetSwitched,29
3,2019-03-30 20:14:41,1112085761569054723,None,None,None,254333914,Hypable,Hypable,254333914,RT @Hypable: 'Avengers: Endgame' posters have ...,1377394638,Christian_sexy7,266
4,2019-03-30 20:14:41,1112085762030358530,MTV_iwan,1112085505330634752,852572866512027648,None,None,MTV_iwan,852572866512027650,@MTV_iwan @theflairqueenx @BeckyLynchWWE I gue...,1105100693147394048,QuincyWard15,28


Now, it's time to initialize the Graph. We can do this by calling the function .Graph() of NetworkX.

In [16]:
graph = nx.Graph()

There are two other important functions to create a Graph. The first one is add_node()and the second one, add_edge both with a very descriptive name. Let's pay attention to the syntax of add_edge:  

`Graph.add_edge(u_of_edge, v_of_edge, **attr)`  

where `u`, `v` are the nodes, and attr are keyword arguments that characterize the edge data such as weight, capacity, length, etc.
  
If we add an edge that already exists, the edge data will get updated. Also, if we are an edge between two nodes that are still not in the Graph, the nodes will be created in the process.
We are going to populate the Graph by calling the function get_interactions that we defined earlier. With this information, we apply the function add_edge to every tuple consisting of the tweet's user_id and the user_id of the user mentioned, replied to or retweeted, creating the nodes and the edges connecting them. Also, the tweet id will be added as edge data.

In [17]:
for index, tweet in tweets_final.iterrows():
    user, interactions = get_interactions(tweet)
    user_id, user_name = user
    tweet_id = tweet["id"]
    #tweet_sent = tweet["sentiment"]
    for interaction in interactions:
        int_id, int_name = interaction
        graph.add_edge(user_id, int_id, tweet_id=tweet_id)
        

        graph.node[user_id]["name"] = user_name
        graph.node[int_id]["name"] = int_name       

Now that we have the node and edge of the Graph created, let's see the number of nodes and edges present:

In [95]:
f"There are {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges present in the Graph"

'There are 3808 nodes and 3174 edges present in the Graph'

In [67]:
degrees = [val for (node, val) in graph.degree()]

In [51]:
max(degrees)

485

In [52]:
min(degrees)

1

In [76]:
#degrees_count = [(val, degrees.count(val)) for val in set(degrees)]

In [35]:
#positions = nx.spring_layout(graph)

In [87]:
#subgraphs = list(nx.connected_component_subgraphs(graph, copy=False))

In [114]:
#subgraphs_final = [sub for sub in subgraphs if len(sub)>50]

In [122]:
#len(subgraphs_final)

In [121]:
#nx.draw(subgraphs_final[0], nodesize=1)

In [ ]:
#pg = nx.pagerank(graph)

In [ ]:
#plt.figure(figsize = (20,20))
#nx.draw(graph, width=0.5, arrows=False)
#plt.show()